### SODA and Datalink System Test Notebook

This notebook does some basic queries against the ObsCore table in the TAP service, then uses the datalink service to load the data for the full image, as well as the information about the SODA cutout service to get a specified cutout of the original image.  Both of these images are loaded in this notebook so they can be looked at side-by-side to ensure one is a cutout of the other.

Import required libraries, get a TAP service search object, and set up the notebook to display images.

In [ ]:
import random

from lsst.rsp import get_tap_service
from pyvo.dal.adhoc import DatalinkResults, SodaQuery
from astropy.io import fits
from astropy.utils.data import download_file
from lsst.afw.image import ImageF

import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage
from astropy.visualization import simple_norm, imshow_norm
from astropy.visualization import ImageNormalize,  ZScaleInterval
from astropy.visualization.stretch import SinhStretch, LinearStretch, SqrtStretch

afwDisplay.setDefaultBackend('matplotlib')
service = get_tap_service()

If you already know the butler ID, here is how  you can write a query to query against a butler UUID instead of just a random image.

A couple of UUIDs have been provided as an example.

Note: not all rows in the obscore table can have a cutout taken of them.  They have to be the right type of image as well, such as a deep coadd.

In [ ]:
# Return the first calexp found in the ObsCore table that looks like the ID we have 
#calexp_id = "7fbea78d-4228-4b6a-9386-229e292e0f69"
calexp_id = "7cfcde02-ff73-4eeb-a765-67d04b241667"

query = f"""
SELECT * FROM ivoa.ObsCore WHERE access_url like '%{calexp_id}' 
"""
print(query)


Here we use a query to return 100 random rows from the Obscore table that should be able to get cutouts for each of the images.  Don't run this cell if you are trying to use just one UUID, as it will overwrite the query from the above cell.

In [ ]:
# Let's pick one at random
query = """
SELECT * FROM ivoa.ObsCore where dataproduct_subtype = 'lsst.deepCoadd_calexp' LIMIT 100
"""

Run the query against the TAP service to get the resulting rows:

In [ ]:
# Or let's retrieve the image with a nice galaxy cluster above
results = service.search(query)

Let's show all the resulting rows from the query so you can see the object ID, dataproduct_subtype, ra, dec, and other useful information in the ObsCore table

In [ ]:
results.to_table()

Pick a random row out of the rows that were returned, and get the URL to the datalink service referring to this row.

In [ ]:
# Extract the access URL from the result in the first row
result = random.choice(results)
f"Datalink link service url: {result.getdataurl()}"

From the datalink service URL, we can use the DatalinkResults class to load the XML document from the datalink service URL and show it in a table.  This will show all the related datalinks for that particular row.  In these cases, there should be a raw image URL, as well as a reference to the cutout service.

In [ ]:
dl_results = DatalinkResults.from_result_url(result.getdataurl(),session=service._session)
dl_results.to_table().show_in_notebook()

First, let's look at the first row in the datalinks result: the raw image URL.  This is a google URL to the full image.

In [ ]:
# Full image of calexp - not a cutout
image_url = dl_results.getrecord(0).get('access_url')
image_url

Let's download the full image now, which will be stored in filename.

In [ ]:
# Now let's download the image 
filename = download_file(image_url)
print(filename)

Now let's download the fits headers for this URL...

In [ ]:
hdulist = fits.open(image_url)

for hdu in hdulist:
    print(hdu.name)

Now let's plot the image using matplotlib...

In [ ]:
# Let's plot the image and see what it looks like...
image = hdulist[1].data

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
im = imshow_norm(image, ax, origin='lower', interval=ZScaleInterval(), stretch=SqrtStretch(), cmap='gray')
fig.colorbar(im[0])

Now let's create a SODA request based on that datalink result.  We are using the "cutout-sync" service, which is the SODA service.

We put in the center of the image as the center of the circle, and a smaller radius to make a circle cutout.  Even though this doesn't end up in a circle shape, the resulting image has the whole circle centered at the center of the image.

In [ ]:
sq = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                             session=service._session)
radius = 0.05
sq.circle = (result["s_ra"], result["s_dec"], radius)
sq

This bit makes a filename to download into, and then executes the SODA request, reading all the resulting bytes and writing them into the filename we just created.

In [ ]:
# Download the cutout
sodaPoly = os.path.join(os.getenv('HOME'), 'DATA/soda-polygon.fits')
with open(sodaPoly, 'bw') as f:
    f.write(sq.execute_stream().read())

This takes the cutout image we downloaded and displays it.  You can relate it to the image above to make sure it is indeed a cutout of the larger image.

In [ ]:
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(ImageF(sodaPoly))